# Microarray Analysis using Python

## Data

* file: <code>samplefile.xslx</code>
* Data: brain region microarray data (measures gene expression levels in a tissue)
* 3 experimental variables => 8 experimental groups
    * tissue type: hippocampus & cerebral cortex
    * age: young vs. old
    * treatment: transcranial magnetic stimulation vs control (no treatment)
* Columns: 4 repeats for each experimental group except 1 ... (8 x 4) - 1 = 31 columns in spreadsheet
* Rows: 1 row per read; 1 or more rows per gene. Redundant read need to be merged.

## Method

1. Import file
2. For a given gene with redundant reads, take maximum value (collapse multiple rows into one row)
3. Log transform
4. Z-score
5. For each comparison, take difference of means of repeats and pvalue
6. Create ribbon plot for genes with fold change of 1 and p-value < 0.05

# Load some Python libraries used for spreadsheet stuff

In [1]:
import pandas as pd
import numpy as np

# Read in Excel File

In [2]:
df = pd.read_excel( 'samplefile.xlsx')

# See what we got

In [3]:
df

ArrayID        Symbol  AVG_Signal_BR1_TEST_O_1  \
0            1           NA1             49209.040000   
1            2           NA2                 2.463345   
2            3           NA3                 2.481884   
3            4       Tbc1d19               772.916500   
4            5          Cfc1                24.287960   
5            6         Foxo3                90.880950   
6            7         Spag9               375.965600   
7            8           NA4             31344.580000   
8            9    RGD1311899             40745.100000   
9           10        Olr390               258.404700   
10          11         Nabp2              1609.616000   
11          12        Olr783                 2.601339   
12          13          Dok2                23.419990   
13          14           NA5              3126.112000   
14          15          Ist1             12126.560000   
15          16           NA6                 2.947477   
16          18         Obfc1              1163.352000   
17          19         Prepl              2451.280000   
18          20        Pcdhb8               151.617700   
19          21           NA7                 2.644272   
20          22          Dstn              9734.927000   
21          23           NA8                 2.645163   
22          24       Olr1421                 2.644604   
23          25          Grm1              6755.685000   
24          26  LOC102549471              2021.165000   
25          27        Supt7l               499.694400   
26          28       Fam91a1               181.824100   
27          29    RGD1559917              7772.458000   
28          30         Apba2              2940.061000   
29          31  LOC103694165                 2.624371   
...        ...           ...                      ...   
59704    62943          Figf                 2.592219   
59705    62946       Olr1245                 2.612217   
59706    62948         Bcl6b               186.302700   
59707    62949         Taco1               267.356100   
59708    62950      RT1-M1-2               181.666900   
59709    62951         Zc4h2               358.620900   
59710    62952        Trim29                 8.711253   
59711    62953     Krtap13-2                19.587200   
59712    62954        Defb36               129.775000   
59713    62955          Vwa1              4804.037000   
59714    62956       NA11367                 2.650564   
59715    62957        Gtf3c4                12.417430   
59716    62958       Rnase10                71.158780   
59717    62959      Tmprss15                 2.650936   
59718    62960          Brox              3670.129000   
59719    62961          Cln5               943.194900   
59720    62962       NA11368               198.567200   
59721    62963       NA11369                 4.300480   
59722    62965            C2               801.553800   
59723    62966         Rhox3                 2.624992   
59724    62967          Fhl1              7544.378000   
59725    62968          Evx1                 6.773881   
59726    62969  LOC100912076                73.063350   
59727    62970          Dtd1              2891.303000   
59728    62971       NA11370                13.796650   
59729    62972  LOC100911030                54.224690   
59730    62973       NA11371                 2.554798   
59731    62974       NA11372                 2.540203   
59732    62975       NA11373             46838.700000   
59733    62976       NA11374             49877.940000   

       AVG_Signal_BR1_TEST_O_2  AVG_Signal_BR1_TEST_O_3  \
0                 55571.090000             28678.790000   
1                     3.040068                 2.177575   
2                     3.074572                 2.201559   
3                   631.279700               584.113500   
4                     3.138412                 3.287079   
5                    68.767660                90.506310   
6                   321.297600               466.173500   


In [4]:
df.shape

(59734, 33)

In [5]:
len( df )

59734

In [6]:
df.columns

Index(['ArrayID', 'Symbol', 'AVG_Signal_BR1_TEST_O_1',
       'AVG_Signal_BR1_TEST_O_2', 'AVG_Signal_BR1_TEST_O_3',
       'AVG_Signal_BR1_TEST_O_4', 'AVG_Signal_BR1_TEST_Y_1',
       'AVG_Signal_BR1_TEST_Y_2', 'AVG_Signal_BR1_TEST_Y_3',
       'AVG_Signal_BR1_TEST_Y_4', 'AVG_Signal_BR1_CONTROL_O_1',
       'AVG_Signal_BR1_CONTROL_O_2', 'AVG_Signal_BR1_CONTROL_O_3',
       'AVG_Signal_BR1_CONTROL_Y_1', 'AVG_Signal_BR1_CONTROL_Y_2',
       'AVG_Signal_BR1_CONTROL_Y_3', 'AVG_Signal_BR1_CONTROL_Y_4',
       'AVG_Signal_BR2_TEST_O_1', 'AVG_Signal_BR2_TEST_O_2',
       'AVG_Signal_BR2_TEST_O_3', 'AVG_Signal_BR2_TEST_O_4',
       'AVG_Signal_BR2_TEST_Y_1', 'AVG_Signal_BR2_TEST_Y_2',
       'AVG_Signal_BR2_TEST_Y_3', 'AVG_Signal_BR2_TEST_Y_4',
       'AVG_Signal_BR2_CONTROL_O_1', 'AVG_Signal_BR2_CONTROL_O_2',
       'AVG_Signal_BR2_CONTROL_O_3', 'AVG_Signal_BR2_CONTROL_O_4',
       'AVG_Signal_BR2_CONTROL_Y_1', 'AVG_Signal_BR2_CONTROL_Y_2',
       'AVG_Signal_BR2_CONTROL_Y_3', 'AVG_Signal_BR2

# Get basic statistics across all columns

In [7]:
df.describe()

ArrayID  AVG_Signal_BR1_TEST_O_1  AVG_Signal_BR1_TEST_O_2  \
count  59734.00000             59734.000000             59734.000000   
mean   31524.80296              5384.473694              4454.449037   
std    18161.78588             17746.062167             15169.345725   
min        1.00000                 2.229295                 2.771278   
25%    15839.25000                23.196915                18.948712   
50%    31507.50000               398.450550               317.348100   
75%    47259.75000              3122.402000              2436.539750   
max    62976.00000            382463.400000            378868.700000   

       AVG_Signal_BR1_TEST_O_3  AVG_Signal_BR1_TEST_O_4  \
count             59734.000000             59734.000000   
mean               4392.878880              3313.541884   
std               14727.989566             11923.159110   
min                   2.021512                 1.943038   
25%                  20.337463                13.306112   
50%                 351.572750               233.752000   
75%                2519.200750              1765.736750   
max              349050.400000            313783.700000   

       AVG_Signal_BR1_TEST_Y_1  AVG_Signal_BR1_TEST_Y_2  \
count             59734.000000             59734.000000   
mean               3697.709862              4002.170351   
std               12977.262288             13535.652161   
min                   3.449527                 3.805483   
25%                  17.246192                18.211133   
50%                 278.701550               303.142700   
75%                2018.656000              2264.094750   
max              333269.100000            333858.500000   

       AVG_Signal_BR1_TEST_Y_3  AVG_Signal_BR1_TEST_Y_4  \
count             59734.000000             59734.000000   
mean               3405.605604              4411.968189   
std               11763.641335             14802.609921   
min                   3.583699                 3.706025   
25%                  16.353920                22.935052   
50%                 258.650200               340.194800   
75%                1915.699750              2509.857750   
max              318876.800000            358140.600000   

       AVG_Signal_BR1_CONTROL_O_1             ...              \
count                59734.000000             ...               
mean                  4124.550038             ...               
std                  14298.382559             ...               
min                      2.009047             ...               
25%                     18.610328             ...               
50%                    306.062350             ...               
75%                   2258.906000             ...               
max                 361630.500000             ...               

       AVG_Signal_BR2_TEST_Y_3  AVG_Signal_BR2_TEST_Y_4  \
count             59734.000000             59734.000000   
mean               3634.197896              2699.198990   
std               12547.142010              9938.363168   
min                   1.891375                 1.788280   
25%                  13.959275                10.805488   
50%                 264.659000               188.446250   
75%                2032.339500              1418.294500   
max              369675.000000            336430.800000   

       AVG_Signal_BR2_CONTROL_O_1  AVG_Signal_BR2_CONTROL_O_2  \
count                59734.000000                59734.000000   
mean                  4492.407612                 5076.130188   
std                  15236.455061                16459.849274   
min                      2.182025                    1.877310   
25%                     19.528125                   19.428725   
50%                    371.489100                  391.695600   
75%                   2526.683500                 2877.560500   
max                 433668.200000               409942.700000   

       AVG_Signal_BR2_CONTROL_O_3  AVG_Signal_BR2_CONTROL_O_4 

# Get basic stats on a particular column

In [8]:
df.AVG_Signal_BR1_TEST_O_2.describe()

count     59734.000000
mean       4454.449037
std       15169.345725
min           2.771278
25%          18.948712
50%         317.348100
75%        2436.539750
max      378868.700000
Name: AVG_Signal_BR1_TEST_O_2, dtype: float64

# How many unique gene symbols do we have?

In [9]:
len(df.Symbol.unique())

29784

# Group rows by gene symbol

In [10]:
grouped = df.groupby('Symbol')

## See what we got from the grouping operation

In [11]:
a_symbol, symbol_df = next(iter(grouped) )

In [12]:
symbol_df

ArrayID Symbol  AVG_Signal_BR1_TEST_O_1  AVG_Signal_BR1_TEST_O_2  \
25901    27344   A1bg                10.149650                 8.174033   
45433    47917   A1bg                 7.784694                 6.597327   

       AVG_Signal_BR1_TEST_O_3  AVG_Signal_BR1_TEST_O_4  \
25901                 6.377124                 3.465803   
45433                 3.413540                 3.383401   

       AVG_Signal_BR1_TEST_Y_1  AVG_Signal_BR1_TEST_Y_2  \
25901                 3.662676                13.010890   
45433                 4.068976                 4.711912   

       AVG_Signal_BR1_TEST_Y_3  AVG_Signal_BR1_TEST_Y_4  \
25901                 8.607907                 4.048179   
45433                 4.278833                 7.669490   

                  ...              AVG_Signal_BR2_TEST_Y_3  \
25901             ...                             2.060191   
45433             ...                             2.376990   

       AVG_Signal_BR2_TEST_Y_4  AVG_Signal_BR2_CONTROL_O_1  \
25901                 3.300403                    3.724781   
45433                 2.262671                    8.824682   

       AVG_Signal_BR2_CONTROL_O_2  AVG_Signal_BR2_CONTROL_O_3  \
25901                    2.081313                    2.378654   
45433                    2.618769                    2.592154   

       AVG_Signal_BR2_CONTROL_O_4  AVG_Signal_BR2_CONTROL_Y_1  \
25901                    2.759006                    3.550204   
45433                    2.472625                    2.389171   

       AVG_Signal_BR2_CONTROL_Y_2  AVG_Signal_BR2_CONTROL_Y_3  \
25901                    2.835767                    2.250051   
45433                    2.459102                    4.228176   

       AVG_Signal_BR2_CONTROL_Y_4  
25901                    4.404741  
45433                    2.538803  

[2 rows x 33 columns]

# Combine reads

Take maximum expression level for a given gene.

In [13]:
max_expression = grouped.max()

# Log transform

In [14]:
log_transformed = max_expression.apply( np.log )

# Zscore

In [15]:
from scipy.stats import zscore

In [16]:
normalized = log_transformed.apply(zscore)

# What comparisons are we interested in?

In [17]:
comparisons = [('BR1_CONTROL_O', 'BR1_CONTROL_Y'),
               ('BR1_TEST_O',    'BR1_TEST_Y' ),
               ('BR2_CONTROL_O', 'BR2_CONTROL_Y'),
               ('BR2_TEST_O',    'BR2_TEST_Y' ),
               
               ('BR1_CONTROL_Y', 'BR1_TEST_Y'),
               ('BR1_CONTROL_O', 'BR1_TEST_O' ),
               ('BR2_CONTROL_Y', 'BR2_TEST_Y'),
               ('BR2_CONTROL_O', 'BR2_TEST_O' ),

               ('BR1_CONTROL_O', 'BR2_CONTROL_O'),
               ('BR1_TEST_O',    'BR2_TEST_O' ),
               ('BR1_CONTROL_Y', 'BR2_CONTROL_Y'),
               ('BR1_TEST_Y',    'BR2_TEST_Y') ]

# Get gene expression difference and p-value for desired comparisons

Use Wilcoxon rank-sum statistic for two samples to test if the reads for a given gene across comparison groups come from different distributions.

In [18]:
from scipy.stats import ranksums

In [19]:
import time

In [20]:
all_diffs = {}
all_pvals = {}

for groupa, groupb in [ list(_) for _ in comparisons ]:
    
    t1 = time.time()
    comp_name = groupa + '_vs_' + groupb
    
    # First: select only the columns that are involvoled in this comparison
    groupa_col_names = [ _ for _ in normalized.columns if groupa in _ ]
    groupb_col_names = [ _ for _ in normalized.columns if groupb in _ ]
    groupa_cols = normalized[ groupa_col_names ]
    groupb_cols = normalized[ groupb_col_names ]
    
    # Second: do diffs for all genes in one step:
    all_diffs[ comp_name ] = groupa_cols.mean(axis=1) - groupb_cols.mean(axis=1)
    
    # Third: go row-by-row doing significance test
    pval_list = []
    for groupa_vals, groupb_vals in zip( groupa_cols.as_matrix(), groupb_cols.as_matrix() ):
        statistic, pvalue = ranksums( groupa_vals, groupb_vals ) 
        pval_list.append( pvalue )
    all_pvals[ comp_name ] = pval_list
    
    # Fourth: say how long it took
    t2 = time.time()
    print( "{} completed in {:0.2f} seconds".format( comp_name, t2-t1 ))
    t1 = t2
    
all_diffs = pd.DataFrame.from_dict( all_diffs )
all_pvals = pd.DataFrame.from_dict( all_pvals )
all_diffs.index = all_pvals.index = normalized.index
print( "FINISHED")

BR1_CONTROL_O_vs_BR1_CONTROL_Y completed in 5.09 seconds
BR1_TEST_O_vs_BR1_TEST_Y completed in 5.04 seconds
BR2_CONTROL_O_vs_BR2_CONTROL_Y completed in 4.92 seconds
BR2_TEST_O_vs_BR2_TEST_Y completed in 4.71 seconds
BR1_CONTROL_Y_vs_BR1_TEST_Y completed in 4.98 seconds
BR1_CONTROL_O_vs_BR1_TEST_O completed in 4.69 seconds
BR2_CONTROL_Y_vs_BR2_TEST_Y completed in 4.90 seconds
BR2_CONTROL_O_vs_BR2_TEST_O completed in 5.13 seconds
BR1_CONTROL_O_vs_BR2_CONTROL_O completed in 5.15 seconds
BR1_TEST_O_vs_BR2_TEST_O completed in 5.02 seconds
BR1_CONTROL_Y_vs_BR2_CONTROL_Y completed in 4.36 seconds
BR1_TEST_Y_vs_BR2_TEST_Y completed in 4.28 seconds
FINISHED


# Number of genes with fold change > 1

In [21]:
(all_diffs.abs() > 1).sum()

BR1_CONTROL_O_vs_BR1_CONTROL_Y     0
BR1_CONTROL_O_vs_BR1_TEST_O        0
BR1_CONTROL_O_vs_BR2_CONTROL_O    17
BR1_CONTROL_Y_vs_BR1_TEST_Y        1
BR1_CONTROL_Y_vs_BR2_CONTROL_Y    38
BR1_TEST_O_vs_BR1_TEST_Y           0
BR1_TEST_O_vs_BR2_TEST_O          28
BR1_TEST_Y_vs_BR2_TEST_Y          34
BR2_CONTROL_O_vs_BR2_CONTROL_Y     2
BR2_CONTROL_O_vs_BR2_TEST_O        0
BR2_CONTROL_Y_vs_BR2_TEST_Y        2
BR2_TEST_O_vs_BR2_TEST_Y           0
dtype: int64

# Number of genes with statistically significant different distributions

In [22]:
(all_pvals < 0.05).sum()

BR1_CONTROL_O_vs_BR1_CONTROL_Y     5113
BR1_CONTROL_O_vs_BR1_TEST_O        1065
BR1_CONTROL_O_vs_BR2_CONTROL_O     9633
BR1_CONTROL_Y_vs_BR1_TEST_Y        7960
BR1_CONTROL_Y_vs_BR2_CONTROL_Y    10877
BR1_TEST_O_vs_BR1_TEST_Y           7739
BR1_TEST_O_vs_BR2_TEST_O           9929
BR1_TEST_Y_vs_BR2_TEST_Y          12887
BR2_CONTROL_O_vs_BR2_CONTROL_Y     7700
BR2_CONTROL_O_vs_BR2_TEST_O         842
BR2_CONTROL_Y_vs_BR2_TEST_Y        2500
BR2_TEST_O_vs_BR2_TEST_Y           2932
dtype: int64

# Both

In [23]:
((all_diffs.abs() > 1) & (all_pvals < 0.05)).sum()

BR1_CONTROL_O_vs_BR1_CONTROL_Y     0
BR1_CONTROL_O_vs_BR1_TEST_O        0
BR1_CONTROL_O_vs_BR2_CONTROL_O    17
BR1_CONTROL_Y_vs_BR1_TEST_Y        0
BR1_CONTROL_Y_vs_BR2_CONTROL_Y    36
BR1_TEST_O_vs_BR1_TEST_Y           0
BR1_TEST_O_vs_BR2_TEST_O          28
BR1_TEST_Y_vs_BR2_TEST_Y          34
BR2_CONTROL_O_vs_BR2_CONTROL_Y     0
BR2_CONTROL_O_vs_BR2_TEST_O        0
BR2_CONTROL_Y_vs_BR2_TEST_Y        0
BR2_TEST_O_vs_BR2_TEST_Y           0
dtype: int64

# Focus on one comparison: BR1_CONTROL_Y_vs_BR2_CONTROL_Y

In [24]:
exp_diffs = all_diffs['BR1_CONTROL_Y_vs_BR2_CONTROL_Y']
exp_pvals = all_pvals['BR1_CONTROL_Y_vs_BR2_CONTROL_Y']

In [25]:
plot_these = exp_diffs.loc[ (exp_diffs.abs() > 1) & (exp_pvals < 0.05) ].sort_values()

In [26]:
plot_these = plot_these.to_frame( name='zscore' )

Turn the row labels into variables in their own right:

In [27]:
plot_these['Gene'] = plot_these.index

# Generate RibbonPlot

## Load some of Python's figure-making libraries:

In [28]:
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Tell Python to show the figure after it's done making it:

In [29]:
%matplotlib inline

## Make the figure:

In [32]:
sns.set( style="whitegrid" )
figure, axes = plt.subplots( figsize=(6, 10) )
sns.barplot( data=plot_these, x="zscore", y="gene" )
chart_title = """Normalized difference in gene expression
Hippocampus vs. Cerebral Cortex
(Young Control samples)"""
axes.set_title( chart_title, size=18 )
axes.set_xlabel( "Fold change")

## Save the figure as a PDF:

In [31]:
figure.savefig( "ribbonplot.pdf")

# DONE!

# On your own:


In [5]:
Suv420h1 = df[df.Symbol == 'Suv420h1']

In [19]:
Suv420h1.columns

Index(['ArrayID', 'Symbol', 'AVG_Signal_BR1_TEST_O_1',
       'AVG_Signal_BR1_TEST_O_2', 'AVG_Signal_BR1_TEST_O_3',
       'AVG_Signal_BR1_TEST_O_4', 'AVG_Signal_BR1_TEST_Y_1',
       'AVG_Signal_BR1_TEST_Y_2', 'AVG_Signal_BR1_TEST_Y_3',
       'AVG_Signal_BR1_TEST_Y_4', 'AVG_Signal_BR1_CONTROL_O_1',
       'AVG_Signal_BR1_CONTROL_O_2', 'AVG_Signal_BR1_CONTROL_O_3',
       'AVG_Signal_BR1_CONTROL_Y_1', 'AVG_Signal_BR1_CONTROL_Y_2',
       'AVG_Signal_BR1_CONTROL_Y_3', 'AVG_Signal_BR1_CONTROL_Y_4',
       'AVG_Signal_BR2_TEST_O_1', 'AVG_Signal_BR2_TEST_O_2',
       'AVG_Signal_BR2_TEST_O_3', 'AVG_Signal_BR2_TEST_O_4',
       'AVG_Signal_BR2_TEST_Y_1', 'AVG_Signal_BR2_TEST_Y_2',
       'AVG_Signal_BR2_TEST_Y_3', 'AVG_Signal_BR2_TEST_Y_4',
       'AVG_Signal_BR2_CONTROL_O_1', 'AVG_Signal_BR2_CONTROL_O_2',
       'AVG_Signal_BR2_CONTROL_O_3', 'AVG_Signal_BR2_CONTROL_O_4',
       'AVG_Signal_BR2_CONTROL_Y_1', 'AVG_Signal_BR2_CONTROL_Y_2',
       'AVG_Signal_BR2_CONTROL_Y_3', 'AVG_Signal_BR2

In [7]:
from scipy.stats.mstats import kruskalwallis

In [17]:
Suv420h1[ Suv420h1.columns[2:]].as_matrix().T

array([[ 3489.667 ,   342.7536,  3543.772 ,  3480.075 ,   360.5167,
          354.0101,   377.8154,  3334.497 ,   361.9391,   382.2341,
          363.5269,   333.4587,  3483.586 ,  3327.763 ,   413.9088,
         3312.904 ,   398.6894,  3274.656 ,  3620.349 ,  3227.726 ],
       [ 2639.498 ,   282.0036,  2632.292 ,  2601.357 ,   328.7193,
          314.7441,   294.0218,  2573.176 ,   317.2552,   331.4735,
          316.1163,   308.6923,  2516.118 ,  2507.476 ,   329.4245,
         2617.768 ,   311.4156,  2480.783 ,  2560.056 ,  2525.45  ],
       [ 2876.917 ,   379.509 ,  3024.015 ,  2937.448 ,   377.0802,
          365.8818,   392.6335,  2826.608 ,   371.7557,   401.4565,
          368.5963,   379.8986,  2878.745 ,  2886.556 ,   387.0985,
         2669.294 ,   375.2104,  2904.073 ,  2886.647 ,  2863.437 ],
       [ 1851.872 ,   241.7938,  1935.529 ,  1825.163 ,   234.9722,
          234.1275,   270.086 ,  1825.918 ,   237.1046,   235.4798,
          258.981 ,   213.4933,  1798.143 ,  

In [15]:
?kruskalwallis

In [18]:
kruskalwallis( *Suv420h1[ Suv420h1.columns[2:]].as_matrix().T)

KruskalResult(statistic=138.8275486987684, pvalue=6.1623592332344918e-16)

In [20]:
br2_control_Y_cols = ['AVG_Signal_BR2_CONTROL_Y_1', 'AVG_Signal_BR2_CONTROL_Y_2',
       'AVG_Signal_BR2_CONTROL_Y_3', 'AVG_Signal_BR2_CONTROL_Y_4']

In [21]:
kruskalwallis( *Suv420h1[ br2_control_Y_cols ].as_matrix().T)

KruskalResult(statistic=2.5916666666666686, pvalue=0.45895236119240346)

In [22]:
br2_control_O_cols = ['AVG_Signal_BR2_CONTROL_O_1', 'AVG_Signal_BR2_CONTROL_O_2',
       'AVG_Signal_BR2_CONTROL_O_3', 'AVG_Signal_BR2_CONTROL_O_4']

In [23]:
kruskalwallis( *Suv420h1[ br2_control_O_cols ].as_matrix().T)

KruskalResult(statistic=16.722777777777765, pvalue=0.00080584517919065853)

In [24]:
Suv420h1[ br2_control_O_cols ]

AVG_Signal_BR2_CONTROL_O_1  AVG_Signal_BR2_CONTROL_O_2  \
3988                    2453.9690                   3189.3200   
4232                     307.7886                    410.9277   
4742                    2609.8770                   3139.3770   
4757                    2499.5070                   3152.1180   
5117                     327.8180                    383.2319   
5165                     307.4982                    383.7452   
8677                     327.2883                    388.2755   
10867                   2601.6060                   3130.3290   
15075                    325.2123                    377.3843   
19300                    354.4660                    384.0164   
19398                    335.9415                    423.8230   
21802                    337.6297                    397.3462   
26546                   2497.5180                   3082.8230   
28403                   2480.2910                   2985.8620   
32428                    351.3353                    435.3264   
40216                   2588.9200                   3046.6350   
43490                    328.3722                    400.5328   
45031                   2593.0980                   2979.6200   
45479                   2602.7700                   3217.8780   
59043                   2591.5880                   3053.4550   

       AVG_Signal_BR2_CONTROL_O_3  AVG_Signal_BR2_CONTROL_O_4  
3988                    2118.7540                   1993.3130  
4232                     228.2103                    284.8570  
4742                    1995.2440                   2065.7060  
4757                    2106.1940                   2057.6180  
5117                     238.2811                    313.2470  
5165                     224.7455                    254.8551  
8677                     231.9130                    282.9534  
10867                   1975.7410                   1906.3970  
15075                    242.5061                    263.7204  
19300                    235.1008                    278.3628  
19398                    243.5537                    283.1505  
21802                    240.1587                    273.5666  
26546                   1994.0210                   1948.9950  
28403                   1946.5490                   1917.4320  
32428                    251.3716                    292.2646  
40216                   1896.4570                   1830.3370  
43490                    217.5851                    265.7556  
45031                   2040.5720                   1927.0730  
45479                   2083.4380                   2004.3680  
59043                   1951.6440                   1794.3030